In [15]:
# Esto es para cargar todas las librerias

from ctypes.wintypes import SIZE
import PySimpleGUI as sg
import pandas as pd
from datetime import date
from datetime import datetime
from win32com import client

import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string

from openpyxl.styles import PatternFill, GradientFill
from openpyxl.utils import get_column_letter

from openpyxl import Workbook
from openpyxl.styles import Border, Side

from openpyxl.styles import Alignment
import numpy as np
import sys



In [16]:
# Esto es el codigo para la fecha de expiracion

def verificarfechas():
    expiracion = pd.read_csv(r'C:\Users\XPC\Desktop\software-financiero\Ajustes.csv')
    expiracion = expiracion.values.tolist()
    expiracion = expiracion[0][0]
    expiracion=datetime.strptime(expiracion, '%b-%d-%Y').date()

    hoy = date.today()

    if hoy>expiracion:
        sg.popup("Demo finalizado")
        sys.exit('listofitems not long enough')

verificarfechas()




In [17]:
# Este es el menu derecho que muestra todos los botones para los diferentes modulos

h=5
menubotones= [

    [sg.Button("Sistema de Contabilidad", size=(30,h),pad=(0,0),expand_y=True)],
    [sg.Button("Prónostico de Ventas", size=(30,h),pad=(0,0),expand_y=True, disabled=True)],
    [sg.Button("Manejo de Inventarios", size=(30,h),pad=(0,0),expand_y=True, disabled=True)],
    [sg.Button("Facturación", size=(30,h),pad=(0,0),expand_y=True, disabled=True)],
    [sg.Button("Manejo de Planilla", size=(30,h),pad=(0,0),expand_y=True, disabled=True)],
    [sg.Button("Analísis Financiero", size=(30,h),pad=(0,0),expand_y=True, disabled=True)],
    [sg.Button("Cuentas por pagar y cobrar", size=(30,h),pad=(0,0),expand_y=True, disabled=True)],
    [sg.Button("Ajustes", size=(14,h),pad=(1,0)), sg.Button("Salir", size=(14,h))]

]

In [18]:
# Esto es para generar el catalogo de cuentas y el asiento a registrar

registrodataSA = []
registrodataSA_heads = ['# Asiento','Fecha','Código', 'Cuenta', 'Debe', 'Haber']
#registrodataSA.append(['A', 'B', 'C', 'D'])
registrodataSA_width = [0,0,10, 44, 10, 10]
        
superiorA= [

        [sg.Text("Fecha", size=(9,1)),sg.Text("Asiento #",size=(9,1)),sg.Text("Nombre de la cuenta")],
        [sg.Input(date.today().strftime("%b-%d-%Y"), size=(12,1), key="-FECHA-", disabled=True),sg.Input("", size=(10,1),key="-#ASIENTO-", disabled=True),sg.Input("", size=(71,None),key="-NOMBRECUENTA-", disabled=True)],
        [sg.Text("Código"),sg.Input("", size=(10,None),key="-CODIGO-", disabled=True),sg.Text("Monto"),sg.Input("", size=(35,None), key="-MONTO-"),sg.Radio('Debe', "RADIO1",key="-DEBE-", default=True),sg.Radio('Haber', "RADIO1",key="-HABER-"),sg.Button("Agregar Cuenta", key="-REGISTRARLINEA-")],
        [sg.Button("Editar Entrada", size=(41,2)),sg.Button("Eliminar Entrada", size=(41,2))],
        [sg.Table(values=registrodataSA, headings=registrodataSA_heads,max_col_width=65,col_widths=registrodataSA_width,auto_size_columns=False,justification='left',num_rows=7,background_color="White", row_height=24,text_color="black", key="-ASIENTO-")],
        [sg.Button("Registrar Asiento ", size=(84,2), key="-REGISTRARASIENTO-"),]

]
 

url = 'https://raw.githubusercontent.com/jchavesmartinez/software-financiero/main/cuentas%20-%20Copy.csv'
totalcuentas = pd.read_csv(url)
totalcuentas=totalcuentas.drop(['Nivel', 'Primer Nivel'], axis=1)

registrodataSB=totalcuentas
registrodataSB_heads = ['Código', 'Cuenta']
registrodataSB=registrodataSB.values.tolist()
registrodataSB_width = [9, 49]

superiorB= [

    [sg.Button("Agregar Cuenta", expand_x=True, visible=False),sg.Button("Modificar Cuenta", expand_x=True, visible=False),sg.Button("Eliminar Cuenta", expand_x=True, visible=False)],
    [sg.Text("Buscar por nombre"),sg.Input("",expand_x=True, key="-FILTRARCUENTAS-", enable_events=True)],
    [sg.Table(values=registrodataSB, headings=registrodataSB_heads,max_col_width=65,col_widths=registrodataSB_width,auto_size_columns=False,justification='left',num_rows=11,background_color="White", row_height=25,text_color="black", key="-CUENTAS-", enable_events=True,select_mode=sg.TABLE_SELECT_MODE_BROWSE)],
    [sg.Button("Catalogo Completo", size=(32,2), key="-COMPLETO-"),sg.Button("Catalogo Simplificado", size=(32,2), key="-SIMPLIFICADO-"),]

]

In [19]:
# Esto muestra el historico de todos los asientos registrados asi como la opcion de generar los estados financieros

url = 'https://raw.githubusercontent.com/jchavesmartinez/software-financiero/main/historicoCSV.csv'
registrodataIA = pd.read_csv(r'C:\Users\XPC\Desktop\software-financiero\historicoCSV.csv')
registrodataIA=registrodataIA.values.tolist()
registrodataIA_heads = ['Asiento','Fecha','Código', 'Cuenta', 'Debe', 'Haber']
registrodataIA_width = [6,10,10, 68, 10, 10]

for x in registrodataIA:
   del x[0]

inferiorA= [

    [sg.Table(values=registrodataIA, headings=registrodataIA_heads,max_col_width=65,col_widths=registrodataIA_width,auto_size_columns=False,justification='left',num_rows=9,background_color="White", row_height=25,text_color="black", key="-HISTORICO-")],
    [sg.Button("Realizar cierre de mes", size=(129,4))]
    
]

inferiorB= [

    [sg.Text("Exportar")],
    [sg.Button("Libro Mayor",size=(22,2),pad=(0,0), key="-MAYORPDF-")],
    [sg.Button("Libro Diario",size=(22,2),pad=(0,0), key="-DIARIOPDF-")],
    [sg.Button("Situación Financiera",size=(22,2),pad=(0,0), key="-SITUACIONPDF-")],
    [sg.Button("Balance General",size=(22,2),pad=(0,0), key="-BALANCEPDF-")],
    [sg.Text("",size=(10,6))]

]


In [20]:
# Esto consolida todas las partes de la intefaz en un  solo layout

menucompleto = [
    [sg.Frame("", [[sg.Frame("", superiorA,border_width=0),sg.Frame("", superiorB,border_width=0)]],border_width=0, pad=(0,0))],
    [sg.Frame("", [[sg.Frame("", inferiorA,border_width=0),sg.Frame("", inferiorB,border_width=0)]],border_width=0, pad=(0,0))],
]

menuprincipal = [[sg.Frame("", menubotones,border_width=0)]]

layout = [
    [sg.Frame("", menucompleto,border_width=0,expand_y=True),
     sg.Frame("", menuprincipal,border_width=0,expand_y=True)],]

window= sg.Window("Modulo Contable",layout,finalize=True)


In [21]:
# Esta es la funcion para exportar un Excel a PDF

def exportarpdf(archivoexcel,direccionexportar):
    excel = client.Dispatch("Excel.Application")

    sheets = excel.Workbooks.Open(archivoexcel)
    work_sheets = sheets.Worksheets[0]

    work_sheets.ExportAsFixedFormat(0,direccionexportar)

    sheets.Close()
    #excel.Quit()


In [22]:
# Esto es para crear el Libro Mayor

def LibroMayor():
    MayorData= pd.DataFrame(registrodataIA, columns =registrodataIA_heads)
    MayorData=MayorData.replace('-','0')
    MayorData['Debe'] = MayorData['Debe'].astype(float)
    MayorData['Haber'] = MayorData['Haber'].astype(float)
    MayorData=MayorData.drop(['Asiento'], axis=1)

    Mayor = MayorData.groupby(['Código','Cuenta']).sum()
    Mayor = Mayor.reset_index() 

    Mayor['Debe Saldos'] = np.where(Mayor['Código'].astype(str).str.startswith('11') | Mayor['Código'].astype(str).str.startswith('12') | Mayor['Código'].astype(str).str.startswith('5') | Mayor['Código'].astype(str).str.startswith('9'), Mayor['Debe']-Mayor['Haber'], 0)
    Mayor['Haber Saldos'] = np.where(Mayor['Código'].astype(str).str.startswith('21') | Mayor['Código'].astype(str).str.startswith('22') | Mayor['Código'].astype(str).str.startswith('3') | Mayor['Código'].astype(str).str.startswith('4') | Mayor['Código'].astype(str).str.startswith('8'), Mayor['Haber']-Mayor['Debe'], 0)


    Mayor['Debe BG'] = np.where(Mayor['Código'].astype(str).str.startswith('11') | Mayor['Código'].astype(str).str.startswith('12') | Mayor['Código'].astype(str).str.startswith('551') ,Mayor['Debe Saldos'] , 0)
    Mayor['Haber BG'] = np.where(Mayor['Código'].astype(str).str.startswith('21') | Mayor['Código'].astype(str).str.startswith('22') | Mayor['Código'].astype(str).str.startswith('3') ,Mayor['Haber Saldos'] , 0)

    Mayor['Debe ER'] = np.where(Mayor['Código'].astype(str).str.startswith('9') | Mayor['Código'].astype(str).str.startswith('515') | Mayor['Código'].astype(str).str.startswith('531') ,Mayor['Debe Saldos'] , 0)
    Mayor['Haber ER'] = np.where(Mayor['Código'].astype(str).str.startswith('4') & ~Mayor['Código'].astype(str).str.startswith('79') ,Mayor['Haber Saldos'] , 0)

    Mayor['Debe ER2'] = np.where(Mayor['Código'].astype(str).str.startswith('5') ,Mayor['Debe Saldos'] , 0)
    Mayor['Haber ER2'] = np.where(Mayor['Código'].astype(str).str.startswith('4') & ~Mayor['Código'].astype(str).str.startswith('79') ,Mayor['Haber Saldos'] , 0)

    MayorExportar=Mayor.drop(['Debe ER','Haber ER','Debe ER2','Haber ER2','Debe BG','Haber BG'], axis=1)
    
    MayorExportar['Debe'] = np.where(Mayor['Debe'].astype(str).str.startswith('0') ,"-",Mayor['Debe'])
    MayorExportar['Haber'] = np.where(Mayor['Haber'].astype(str).str.startswith('0') ,"-",Mayor['Haber'])
    MayorExportar['Debe Saldos'] = np.where(Mayor['Debe Saldos'].astype(str).str.startswith('0') ,"-",Mayor['Debe Saldos'])
    MayorExportar['Haber Saldos'] = np.where(Mayor['Haber Saldos'].astype(str).str.startswith('0') ,"-",Mayor['Haber Saldos'])


    MayorExportar.to_excel(r'C:\Users\XPC\Desktop\software-financiero\Mayor.xlsx',sheet_name='Libro Mayor', index=False)

    wb = load_workbook('Mayor.xlsx')
    sheet = wb["Libro Mayor"]

    column_widths = [14,100,17,17,17,17]
    
    for row in MayorExportar:
        for i, cell in enumerate(row):
            if len(column_widths) > i:
                if len(cell) > column_widths[i]:
                    column_widths[i] = len(cell)
            else:
                column_widths += [len(cell)]
        
    for i, column_width in enumerate(column_widths,1):  # ,1 to start at 1
        sheet.column_dimensions[get_column_letter(i)].width = column_width

    thin = Side(border_style="thin", color="00000000")
    double = Side(border_style="double", color="00000000")

    for c in sheet['A1:F1'][0]:
        c.border = Border(bottom=double)
    
    for rows in sheet.iter_rows(min_row=1, max_row=16, min_col=1, max_col=6):
        for cell in rows:
            cell.font = Font(name='Cambria',size=12)
            cell.alignment  = Alignment(horizontal='left', vertical='center')
    
    sheet.sheet_properties.pageSetUpPr.fitToPage = True
    sheet.page_setup.fitToWidth = True

    sheet.page_setup.orientation = sheet.ORIENTATION_LANDSCAPE
    sheet.page_setup.fitToHeight = False
    sheet.page_setup.fitToWidth = 1
    
    wb.save(r'C:\Users\XPC\Desktop\software-financiero\Mayor.xlsx')



In [23]:
# Este es el codigo para crear el libro de diario

def LibroDiario():
    diario= pd.DataFrame (registrodataIA, columns =registrodataIA_heads)
    diario.loc[diario.duplicated(['Asiento']),['Asiento']]=''

    diario.to_excel(r'C:\Users\XPC\Desktop\software-financiero\LibroDiario.xlsx', index=False,sheet_name='Libro de Diario')

    wb = load_workbook('LibroDiario.xlsx')
    sheet = wb["Libro de Diario"]

    #for rows in sheet.iter_rows(min_row=1, max_row=400, min_col=1, max_col=7):
    #    for cell in rows:
    #        cell.fill = PatternFill(start_color='00FFFFFF', end_color='00FFFFFF', fill_type="solid")
    
    column_widths = [10,14,14,120,16,16,16]
    
    for row in diario:
        for i, cell in enumerate(row):
            if len(column_widths) > i:
                if len(cell) > column_widths[i]:
                    column_widths[i] = len(cell)
            else:
                column_widths += [len(cell)]
        
    for i, column_width in enumerate(column_widths,1):  # ,1 to start at 1
        sheet.column_dimensions[get_column_letter(i)].width = column_width

    thin = Side(border_style="thin", color="00000000")
    double = Side(border_style="double", color="00000000")

    for c in sheet['A1:F2'][0]:
        c.border = Border(bottom=double)
    
    for rows in sheet.iter_rows(min_row=1, max_row=16, min_col=1, max_col=8):
        for cell in rows:
            cell.font = Font(name='Cambria',size=12)
            cell.alignment  = Alignment(horizontal='left', vertical='center')


    sheet.sheet_properties.pageSetUpPr.fitToPage = True
    sheet.page_setup.fitToWidth = True

    sheet.page_setup.orientation = sheet.ORIENTATION_LANDSCAPE
    sheet.page_setup.fitToHeight = False
    sheet.page_setup.fitToWidth = 1

    wb.save(r'C:\Users\XPC\Desktop\software-financiero\LibroDiario.xlsx')



In [24]:
# Esto es el codigo para ejecutar la interfaz


while True:
    
    event, values = window.read()

    
    if event == sg.WIN_CLOSED:
        break
    
    if event == '-CUENTAS-'and values['-CUENTAS-']:
        data_selected = [registrodataSB[row] for row in values[event]]
        window['-CODIGO-'].update(data_selected[0][0])
        window['-NOMBRECUENTA-'].update(data_selected[0][1])
        if len(registrodataIA)==0:
            numero=1
        else:
            numero=int(registrodataIA[len(registrodataIA)-1][0])+1
        window['-#ASIENTO-'].update(numero)

    if event == '-SIMPLIFICADO-':
        window['-FILTRARCUENTAS-'].update("")
        cuentasimplificado=totalcuentas.loc[(totalcuentas['Código'] <= 99999)].values.tolist()
        window['-CUENTAS-'].update(cuentasimplificado)
        registrodataSB=cuentasimplificado
    if event == '-COMPLETO-':
        cuentatotal=totalcuentas.loc[(totalcuentas['Código'] <= 9999999999)].values.tolist()
        window['-CUENTAS-'].update(cuentatotal)
        registrodataSB=cuentatotal
    if event == '-FILTRARCUENTAS-' and values['-FILTRARCUENTAS-']:
        filtrarcuentas=totalcuentas.applymap(str)
        filtro=filtrarcuentas[filtrarcuentas.stack().str.contains(values['-FILTRARCUENTAS-']).any(level=0)].values.tolist()
        window['-CUENTAS-'].update(filtro)
        registrodataSB=filtro
    try:
        if event == '-REGISTRARLINEA-' and float(values['-MONTO-'])>0 and values['-NOMBRECUENTA-'] and values['-CODIGO-'] :
            nuevoasiento=[values['-#ASIENTO-'],values['-FECHA-'],data_selected[0][0],data_selected[0][1], values['-MONTO-'] if values['-DEBE-']==True else "-", values['-MONTO-'] if values['-HABER-']==True else "-"]
            registrodataSA.append(nuevoasiento)
            window['-ASIENTO-'].update(registrodataSA)
            window['-ASIENTO-'].set_vscroll_position(1)
    except:
        sg.Popup('No es un asiento válido, por favor revise los campos')
    
    try:              
        if event == '-REGISTRARASIENTO-':        
            debe=[]
            haber=[]

            for x in range(0, len(registrodataSA)):
                debe.append(registrodataSA[x][4])
                haber.append(registrodataSA[x][5])

            debe = [s.replace("-", '0') for s in debe]
            haber = [s.replace("-", '0') for s in haber]

            validacion=sum(map(float, debe))==sum(map(float, haber))

            if validacion==True:
                for x in range(0, len(registrodataSA)):
                    registrodataIA.append(registrodataSA[x])
                window['-HISTORICO-'].update(registrodataIA)
                window['-NOMBRECUENTA-'].update("")
                window['-CODIGO-'].update("")
                window['-MONTO-'].update("")
                registrodataSA = []
                window['-ASIENTO-'].update(registrodataSA)
                if len(registrodataIA)==0:
                    numero=1
                else:
                    numero=int(registrodataIA[len(registrodataIA)-1][0])+1
                window['-#ASIENTO-'].update(numero)
                window['-FECHA-'].update(date.today().strftime("%b-%d-%Y"))
                window['-HISTORICO-'].set_vscroll_position(1)

                pd.DataFrame(registrodataIA, columns = registrodataIA_heads).to_csv("historicoCSV.csv")
            else:
                sg.Popup('No es un asiento válido, por favor revise los campos')

    except:
        sg.Popup('No es un asiento válido, por favor revise los campos') 

    if event == '-DIARIOPDF-':

        LibroDiario()
        archivoexcel= r'C:\Users\XPC\Desktop\software-financiero\LibroDiario.xlsx'
        direccionexportar= r'C:\Users\XPC\Desktop\software-financiero\LibroDiario.pdf'
        exportarpdf(archivoexcel,direccionexportar)
        sg.popup("Libro de diario generado exitosamente")

    if event == '-MAYORPDF-':
        
        LibroMayor()
        archivoexcel= r'C:\Users\XPC\Desktop\software-financiero\Mayor.xlsx'
        direccionexportar= r'C:\Users\XPC\Desktop\software-financiero\Mayor.pdf'
        exportarpdf(archivoexcel,direccionexportar)
        sg.popup("Libro mayor generado exitosamente")

    if event == '-SITUACIONPDF-':
        
        archivoexcel= r'C:\Users\XPC\Desktop\software-financiero\pruebaexcel.xlsx'
        direccionexportar= r'C:\Users\XPC\Desktop\software-financiero\resultado.pdf'
        exportarpdf(archivoexcel,direccionexportar)

    if event == '-BALANCEPDF-':
        
        archivoexcel= r'C:\Users\XPC\Desktop\software-financiero\pruebaexcel.xlsx'
        direccionexportar= r'C:\Users\XPC\Desktop\software-financiero\resultado.pdf'
        exportarpdf(archivoexcel,direccionexportar)


window.close()



